# Processing ICGC Data for Mutational Signatures Analysis

**Goal**: clean the downloaded ICGC donor SSM data and convert them into [mutational annotation format (MAF)](https://docs.gdc.cancer.gov/Data/File_Formats/MAF_Format/), and then convert MAF data into a mutational spectra matrix for mutational signature analysis.

This notebook assumes that the steps in the notebook `Consuming ICGC Data.ipynb` has already been run and the downloaded WGS SSM data is stored in the `/data/WGS/` folder of the project directory.

In [1]:
from pathlib import Path

import pandas as pd


pd.set_option('display.max_columns', None)

In [2]:
dir_data = Path.cwd().parent / "data"
dir_wgs = dir_data / "WGS/"

In [3]:
data = pd.read_csv(dir_wgs / "BRCA-UK_ssm_WGS.tsv.gz", sep="\t")

/var/folders/8l/pbcwnq316rz5v5pnyv_sydmn2fsdlk/T/ipykernel_74605/125577739.py:1: DtypeWarning: Columns (22,23,24,26,27,35,39) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(dir_wgs / "BRCA-UK_ssm_WGS.tsv.gz", sep="\t")


In [4]:
data.head()

,icgc_mutation_id,icgc_donor_id,project_code,icgc_specimen_id,icgc_sample_id,matched_icgc_sample_id,submitted_sample_id,submitted_matched_sample_id,chromosome,chromosome_start,chromosome_end,chromosome_strand,assembly_version,mutation_type,reference_genome_allele,mutated_from_allele,mutated_to_allele,quality_score,probability,total_read_count,mutant_allele_read_count,verification_status,verification_platform,biological_validation_status,biological_validation_platform,consequence_type,aa_mutation,cds_mutation,gene_affected,transcript_affected,gene_build_version,platform,experimental_protocol,sequencing_strategy,base_calling_algorithm,alignment_algorithm,variation_calling_algorithm,other_analysis_algorithm,seq_coverage,raw_data_repository,raw_data_accession,initial_data_release_date
0,MU2050016,DO1007,BRCA-UK,SP2150,SA6146,SA6216,PD4085a,PD4085b,9,38437800,38437800,1,GRCh37,single base substitution,C,C,T,NaN,NaN,33.0,7.0,not tested,NaN,NaN,NaN,intergenic_region,NaN,NaN,NaN,NaN,75.0,Illumina HiSeq,NaN,WGS,NaN,NaN,PCAWG Consensus SNV-MNV caller,NaN,NaN,NaN,FI43481:FI43480,NaN
1,MU2050016,DO1007,BRCA-UK,SP2150,SA6146,SA6216,PD4085a,PD4085b,9,38437800,38437800,1,GRCh37,single base substitution,C,C,T,NaN,NaN,NaN,NaN,not tested,NaN,not tested,NaN,intergenic_region,NaN,NaN,NaN,NaN,75.0,Illumina GA sequencing,NaN,WXS,NaN,BWA http://bio-bwa.sourceforge.net,CaVEMan http://www.nature.com/nature/journal/v...,NaN,NaN,EGA,EGAS00001000161,NaN
2,MU2051169,DO1007,BRCA-UK,SP2150,SA6146,SA6216,PD4085a,PD4085b,8,93543524,93543524,1,GRCh37,single base substitution,C,C,T,NaN,NaN,NaN,NaN,not tested,NaN,not tested,NaN,intergenic_region,NaN,NaN,NaN,NaN,75.0,Illumina GA sequencing,NaN,WXS,NaN,BWA http://bio-bwa.sourceforge.net,CaVEMan http://www.nature.com/nature/journal/v...,NaN,NaN,EGA,EGAS00001000161,NaN
3,MU2051169,DO1007,BRCA-UK,SP2150,SA6146,SA6216,PD4085a,PD4085b,8,93543524,93543524,1,GRCh37,single base substitution,C,C,T,NaN,NaN,54.0,10.0,not tested,NaN,NaN,NaN,intergenic_region,NaN,NaN,NaN,NaN,75.0,Illumina HiSeq,NaN,WGS,NaN,NaN,PCAWG Consensus SNV-MNV caller,NaN,NaN,NaN,FI43481:FI43480,NaN
4,MU92010295,DO1007,BRCA-UK,SP2150,SA6146,SA6216,PD4085a,PD4085b,3,59874611,59874611,1,GRCh37,single base substitution,C,C,T,NaN,NaN,58.0,5.0,not tested,NaN,NaN,NaN,intron_variant,NaN,NaN,ENSG00000189283,ENST00000466788,75.0,Illumina HiSeq,NaN,WGS,NaN,NaN,PCAWG Consensus SNV-MNV caller,NaN,NaN,NaN,FI43481:FI43480,NaN


## Resolving multiple entries for the same mutation in a donor

A simple somatic mutation (SSM) donor dataset in the ICGC portal can contain multiple records for the same variant in a donor. These records differ in fields: `consequence_type`, `aa_mutation`, `cds_mutation`, `gene_affected`, and `transcript_affected`. This is the result of [SnpEff](http://pcingola.github.io/SnpEff/), a genome variant annotation and effect prediction tool.

A single variant can have multiple functional effects (`consequence_type`). One of the reasons is due to the presence of [multiple gene isoforms](https://en.wikipedia.org/wiki/Gene_isoform). These isoforms, while coming from the same locus, can differ in transcription start site, coding DNA sequences, and/or untranslated regions. As a result, [these gene isoforms can have different functions](https://en.wikipedia.org/wiki/Protein_isoform). Sometimes a variant may be transcribed and can introduce synonymous or missense mutation to the transcript. Other times the variant may not be present in the transcript isoform but can influence splice site recognition. Due to these reasons, for the same variant in a donor, we can have multiple `transcript_affected` for the same `gene_affected`.

Additionally, sometimes a variant can exist some distance upstream/downstream of another gene and influence its transcription. As a result, `gene_affected` can also differ for the same variant in a donor.

In [5]:
print("Before cleaning:")

donor_id = "DO1076"

print(f"Donor ID# {donor_id} has {(data.loc[data['icgc_donor_id'] == donor_id].shape[0]):,} records.")
print(f"But they contain only {(data.loc[data['icgc_donor_id'] == donor_id]['icgc_mutation_id'].nunique()):,} unique variants.")

Before cleaning:
Donor ID# DO1076 has 490,103 records.
But they contain only 73,563 unique variants.


In [6]:
select_columns = [
    "icgc_mutation_id", "project_code", "icgc_donor_id",
    "chromosome", "chromosome_start", "chromosome_end",
    "assembly_version", "mutation_type", "reference_genome_allele",
    "mutated_to_allele",
]

In [7]:
data = data[select_columns]

print(f"Data dimensions: {data.shape[0]:,} instances and {data.shape[1]} columns.")

Data dimensions: 1,851,540 instances and 10 columns.


In [8]:
data = data.drop_duplicates(subset=["icgc_donor_id", "icgc_mutation_id"])
data = data.reset_index(drop=True)

print(f"Number of instances after removing multiple records per variant in a donor: {data.shape[0]:,}")

Number of instances after removing multiple records per variant in a donor: 398,988


In [9]:
print("After cleaning:")

donor_id = "DO1076"

print(f"Donor ID# {donor_id} has {(data.loc[data['icgc_donor_id'] == donor_id].shape[0]):,} records.")
print(f"And they contain {(data.loc[data['icgc_donor_id'] == donor_id]['icgc_mutation_id'].nunique()):,} unique variants.")

After cleaning:
Donor ID# DO1076 has 73,563 records.
And they contain 73,563 unique variants.


In [10]:
data.head()

,icgc_mutation_id,project_code,icgc_donor_id,chromosome,chromosome_start,chromosome_end,assembly_version,mutation_type,reference_genome_allele,mutated_to_allele
0,MU2050016,BRCA-UK,DO1007,9,38437800,38437800,GRCh37,single base substitution,C,T
1,MU2051169,BRCA-UK,DO1007,8,93543524,93543524,GRCh37,single base substitution,C,T
2,MU92010295,BRCA-UK,DO1007,3,59874611,59874611,GRCh37,single base substitution,C,T
3,MU2047337,BRCA-UK,DO1007,11,10531779,10531779,GRCh37,single base substitution,C,T
4,MU65416208,BRCA-UK,DO1007,17,46826739,46826739,GRCh37,single base substitution,C,G


## Converting each ICGC SSM dataset into a set of MAF files (one per donor)

In [11]:
def read_ssm_dataset(filepath: Path) -> pd.DataFrame:
    """
    Reads an ICGC SSM file as a pandas dataframe.

    :param filepath: file path to the SSM dataset.
    :return: pandas dataframe selecting only the columns relevant to mutational
        signatures analysis.
    """
    select_columns = [
        "icgc_mutation_id", "project_code", "icgc_donor_id",
        "chromosome", "chromosome_start", "chromosome_end",
        "assembly_version", "mutation_type", "reference_genome_allele",
        "mutated_to_allele",
    ]

    return pd.read_csv(filepath, usecols=select_columns, sep="\t")


def clean_ssm_dataset(data: pd.DataFrame) -> pd.DataFrame:
    """
    Keeps only one variant per donor ID and drops the rest. The repeats are due to
    the SnpEff annotation tool, which is initially irrelevant for signature analysis.

    :param data: a dataframe of SSM.
    :return: a dataframe of SSM without repeats
    """
    return data.drop_duplicates(subset=["icgc_donor_id", "icgc_mutation_id"]).reset_index(drop=True)


def segregate_ids_and_save_as_maf(data: pd.DataFrame,
                                  dir_output: Path) -> None:
    """
    Takes an ICGC SSM dataset, groups them by donor ID, then for each donor ID,
    sorts the records by chromosome number and then by chromosome start position,
    and finally writes this dataset as an MAF file.

    :param data: SSM dataframe
    :param dir_output: output directory for the MAF files.
    """
    for donor_id in data["icgc_donor_id"].unique():
        data_id = data.loc[data["icgc_donor_id"] == donor_id]
        data_id = data_id.loc[pd.to_numeric(data_id["chromosome"], errors="coerce").sort_values().index]
        data_id = data_id.groupby("chromosome", sort=False)\
            .apply(pd.DataFrame.sort_values, "chromosome_start")\
            .reset_index(drop=True)
        data_id.to_csv(dir_output / f"{donor_id}", sep="\t", index=False)


def convert_ssms_to_mafs(dir_datasets: Path, dir_output: Path) -> None:
    """
    Converts each SSM dataset in a directory into MAF files.

    :param dir_datasets: directory containing SSM datasets.
    :param dir_output: directory to store MAF files.
    """
    filepaths = list(dir_datasets.glob("*.tsv.gz"))

    dir_output = dir_output / (dir_datasets.name + "_MAF")
    if not dir_output.exists():
        dir_output.mkdir()

    for filepath in filepaths:
        data = read_ssm_dataset(filepath)
        data = clean_ssm_dataset(data)
        dir_output_file = dir_output / (filepath.stem.split("_")[0])
        if not dir_output_file.exists():
            dir_output_file.mkdir()
        segregate_ids_and_save_as_maf(data, dir_output_file)

In [12]:
convert_ssms_to_mafs(dir_wgs, dir_data)

/var/folders/8l/pbcwnq316rz5v5pnyv_sydmn2fsdlk/T/ipykernel_74605/1962696398.py:16: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(filepath, usecols=select_columns, sep="\t")
